In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture\mine_pub'))

import panel as pn
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')
from bokeh.resources import INLINE
from bokeh import palettes

import paths
import importlib
import functions_plotting as fp
import functions_bondjango as bd
import functions_loaders as fl
import processing_parameters
import numpy as np
import pandas as pd
import h5py
import scipy.stats as stat
import datetime
import umap
from pprint import pprint
from sklearn import metrics
import sklearn.preprocessing as preproc
import sklearn.cluster as cluster
from mine import MineData

In [ ]:
# set up the figure config
importlib.reload(fp)
importlib.reload(processing_parameters)
# define the target saving path
save_path = os.path.join(paths.figures_path, 'TC_regression_interaction')

# define the printing mode
save_mode = True
# define the target document
target_document = 'paper'
# set up the figure theme
fp.set_theme()
# load the label dict
label_dict = processing_parameters.label_dictionary
variable_list = processing_parameters.variable_list
units_dict = processing_parameters.tc_units

In [ ]:
# load the desired files and their associated regressions

# Load the desired files
importlib.reload(processing_parameters)

# load the constants from the regression calculation
time_shifts = processing_parameters.time_shifts
shift_dict = {el: idx for idx, el in enumerate(time_shifts)}
shift_number = len(time_shifts)
# shuffles = processing_parameters.regression_shuffles

# assemble the dataframe columns
reals = ['real_'+str(el) for el in time_shifts]
shuffle_means = ['smean_'+str(el) for el in time_shifts]
columns = reals + shuffle_means + ['mouse', 'day']

# get the search list
search_list = processing_parameters.search_list

# allocate a list for all paths (need to preload to get the dates)
all_paths = []
all_results = []
# for all the search strings
for search_string in search_list:

    # query the database for data to plot
    data_all = bd.query_database('analyzed_data', search_string)
    data_path = [el['analysis_path'] for el in data_all if '_combinedanalysis' in el['slug']]
    data_result = [el['result'] for el in data_all if '_combinedanalysis' in el['slug']]
    all_paths.append(data_path)
    all_results.append(data_result)
# get the dates present
data_dates = np.unique([os.path.basename(el)[:10] for el in np.concatenate(all_paths)])
print(f'Dates present: {data_dates}')

# allocate memory for the resulting dataframe
data = {}
weights = {}
# predictions = {}
day_list = []
animal_list = []
joint_list = []
# for all the list items
for idx0, data_path in enumerate(all_paths):

    # for all the files
    for idx1, files in enumerate(data_path):
        
        # if a habi trial, skip
        if 'habi' in files:
            continue
        
        # get the animal and date from the slug
        name_parts = os.path.basename(files).split('_')
        animal = '_'.join(name_parts[7:10])
        day_s = '_'.join(name_parts[:3])
        day = datetime.datetime.strptime(day_s, '%m_%d_%Y')
        # skip if the animal and day are already evaluated, 
        # since the CC is the same for the whole day
        if animal+'_'+day_s in joint_list:
            continue
        else:
            animal_list.append(animal)
            day_list.append(day)
            joint_list.append(animal+'_'+day_s)
        # assemble the preproc path
        files_preproc = files.replace('_combinedanalysis', '_preproc')
        # open the file
        with pd.HDFStore(files_preproc, 'r') as preprocf:
            if '/cell_matches' in preprocf.keys():
                # get the matches
                cell_matches = preprocf['cell_matches']
                
                print(animal, day_s, files_preproc)
                # get the idx for this file
                current_matches = cell_matches[datetime.datetime.strftime(day, '%m_%d_%Y')].to_numpy()
                current_idx = np.argsort(current_matches).astype(float)
                # remove the nan entries
                current_idx = current_idx[~np.isnan(np.sort(current_matches))]
                # get the roi info
                roi_info = preprocf['roi_info']
        
        # load the data and the cell matches (wasteful, but cleaner I think)
        with h5py.File(files, 'r') as h:
            
            # for all the target variables
            for target_variable in variable_list:
                # create an empty list only if it's the same time this variable runs
                if target_variable not in data.keys():
                    data[target_variable] = []
                    weights[target_variable] = []
                # allocate memory for the real and shuffled regressions
                real_array = np.zeros((shift_number))
                shuffle_array = np.zeros((shift_number))
                real_weight = []
                shuffle_weight = []

                if 'regression' not in h.keys():
                    continue

                # for all the keys (will iterate through shifts and reps for shuffle)
                for key in h['/regression'].keys():

                    # skip if it's not a cc key or is not the target variable
                    if (target_variable not in key) | ('_std' in key):
                        continue
                    # get the time shift 
                    key_parts = key.split('_')
                    shift = int([el[5:] for el in key_parts if 'shift' in el][0])
                    if 'cc' in key:

                        if 'real' in key_parts:
                             # save the values
                            real_array[shift_dict[shift]] = np.array(h['/regression/'+key])
                        else:
                            shuffle_array[shift_dict[shift]] = np.array(h['/regression/'+key])

                    elif ('coefficients' in key and shift == 0):
                        if 'real' in key_parts:
                            real_weight = np.array(h['/regression/'+key])
                        else:
                            shuffle_weight = np.array(h['/regression/'+key])
                    else:
                        continue

                # add the columns to the main list
                data[target_variable].append(list(real_array) + list(shuffle_array) + [animal, day])

                # take only the non time shift (need to check)
                if isinstance(real_weight, list):
                    continue

                temp_df = pd.DataFrame(np.vstack((real_weight, shuffle_weight)).T, columns=['weight', 'shuffle_weight'])
                temp_df['match_id'] = current_idx 
                temp_df['animal'] = animal
                temp_df['day'] = day
                temp_df = pd.concat((temp_df, roi_info), axis=1)
                
                # add a cell id field
                cell_id = np.arange(temp_df.shape[0])
                temp_df['cell_id'] = cell_id

                weights[target_variable].append(temp_df)

            
# for all the variables once more
for target_variable in variable_list:
    # turn the overall list into a dataframe
    data[target_variable] = pd.DataFrame(data[target_variable], columns=columns).sort_values(['mouse', 'day'], axis=0)
    # turn the weights into a dictionary
    weights[target_variable] = pd.concat(weights[target_variable], axis=0)

    print(f'Shape of the data dictionary: {data[target_variable].shape}')
    print(f'Shape of the weights dataframe: {weights[target_variable].shape}')

In [ ]:
%%time
# load the regressions with a loader
importlib.reload(fl)
importlib.reload(processing_parameters)

# get the search list
search_list = processing_parameters.search_list
# get the time shifts 
time_shifts = processing_parameters.time_shifts

# allocate a list for all paths (need to preload to get the dates)
all_paths = []
# all_results = []
# for all the search strings
for search_string in search_list:

    # query the database for data to plot
    data_all = bd.query_database('analyzed_data', search_string)
    data_path = [el['analysis_path'] for el in data_all if '_combinedanalysis' in el['slug']]
#     data_result = [el['result'] for el in data_all if '_combinedanalysis' in el['slug']]
    all_paths.append(data_path)
#     all_results.append(data_result)
# get the dates present
data_dates = np.unique([os.path.basename(el)[:10] for el in np.concatenate(all_paths)])
print(f'Dates present: {data_dates}')

# only load correlations and weights, predictions are not implemented
correlations, regression_df = fl.load_regression(all_paths, variable_list, time_shifts)

In [ ]:
%%time
# load the TCs with a loader
importlib.reload(fl)
importlib.reload(processing_parameters)

# get the search list
search_list = processing_parameters.search_list

# allocate a list for all paths (need to preload to get the dates)
all_paths = []
# all_results = []
# for all the search strings
for search_string in search_list:

    # query the database for data to plot
    data_all = bd.query_database('analyzed_data', search_string)
    data_path = [el['analysis_path'] for el in data_all if '_tcday' in el['slug']]
#     data_result = [el['result'] for el in data_all if '_combinedanalysis' in el['slug']]
    all_paths.append(data_path)
#     all_results.append(data_result)
# get the dates present
data_dates = np.unique([os.path.basename(el)[:10] for el in np.concatenate(all_paths)])
print(f'Dates present: {data_dates}')

# load the tc data
tc_whole = fl.load_tc(all_paths, variable_list)
print(f'Shape of the tuning curve dataframe: {tc_whole.shape}')

In [ ]:
# Assemble the main regression dataframe

# put all the cells in a single dataframe with variables vs cells

# need to iterate through the variables, and concatenate the weights in order per day

# allocate memory for the output dataframe
regression_df = []

# for all the variables
for idx, target_feature in enumerate(weights.keys()):
    # get the data
    current_feature = weights[target_feature]

    if idx == 0:

        df = current_feature.loc[:, ['animal', 'day', 'match_id', 'cell_id', 'weight', 'centroid_x', 'centroid_y',
                                               'bbox_left', 'bbox_top', 'bbox_width', 'bbox_height', 'area']]
        df = df.rename(columns={'weight': target_feature})

    else:
        df = current_feature.loc[:, 'weight']
        df = df.rename(target_feature)

    regression_df.append(df)
# concatenate into a single dataframe
regression_df = pd.concat(regression_df, axis=1)


# eliminate rows with nans
regression_df = regression_df.iloc[~np.any(np.isnan(regression_df.drop(['animal', 'day'], axis=1).to_numpy()), axis=1), :]
print(regression_df.columns)
print(regression_df.shape)
    

In [ ]:
# Load the tc_consolidate file
importlib.reload(processing_parameters)

# get the search query
search_consolidate = processing_parameters.search_consolidate

# query the database for data to plot
data_path = bd.query_database('analyzed_data', search_consolidate)
# data_path = [el['analysis_path'] for el in data_path if 'test' not in el['analysis_path']][0]
data_path = [el['analysis_path'] for el in data_path if 'test' not in el['analysis_path']]
pprint(data_path)

In [ ]:
# Get the relevant fields from the TC data

tc_whole = []
target_features = variable_list
index_columns = ['Resp_index', 'Cons_index', 'Qual_index', 'Resp_test', 'Cons_test', 'Qual_test']
# for all the targets
for idx, target_feature in enumerate(target_features):
    # load the data
    data = []
#     meta_data = []
    for file in data_path:
        
        try:
            data.append(pd.read_hdf(file, target_feature))
            meta_data = pd.read_hdf(file, 'meta_data')
            
            # filter out habi trials
            for idx1, el in enumerate(meta_data.to_numpy()):

                if ('habi' in el):
                    # get a vector with the rows to keep
                    keep_vector = ~(data[-1].loc[:, 'id'] == meta_data.loc[idx1, 'id']).to_numpy()
                    data[-1] = data[-1].iloc[keep_vector, :]

        except KeyError:
            continue
    data = pd.concat(data, axis=0)
    
    # load the relevant columns
    if idx == 0:
        target_columns = ['day', 'animal', 'cell_id'] + index_columns + [el for el in data.columns if ('bin' in el) & ('half' not in el)]
    else:
        target_columns = index_columns + [el for el in data.columns if ('bin' in el) & ('half' not in el)]
    
    data = data.loc[:, target_columns]
    
#     if idx == 0:
#         # convert the day from string to date format
#         data.loc[:, 'day'] = np.array([datetime.datetime.strptime(el, '%m_%d_%Y') for el in data.loc[:, 'day']])
    
    # change the column names
    new_names = {el: target_feature+'_'+el if ('bin' in el) | ('index' in el) | ('test' in el) else el for el in target_columns}
    data = data.rename(columns=new_names)
    # save in the list
    tc_whole.append(data)

# concatenate    
tc_whole = pd.concat(tc_whole, axis=1).reset_index(drop=True)
# print(tc_whole['day'])
# reformat the date
# tc_whole.loc[:, 'day'] = np.array([datetime.datetime.strptime(el, '%m_%d_%Y') for el in tc_whole.loc[:, 'day']])

print(tc_whole.shape)
# exclude all rows with nans
cleanup_columns = [el for el in tc_whole.columns if 'Resp' in el]
cleanup_data = tc_whole.loc[:, cleanup_columns].to_numpy()
cleanup_data[np.isnan(cleanup_data)] = 0
cleanup_data[np.isinf(cleanup_data)] = 0
tc_whole.loc[:, cleanup_columns] = cleanup_data

print(tc_whole.shape)


In [ ]:
# sort the TCs to match the regressions
print(tc_whole.columns)

tc_whole = tc_whole.sort_values(['animal', 'day', 'cell_id'], axis=0).reset_index(drop=True)
# trim the regression(only for this)
regression_df = regression_df.iloc[((regression_df['rvs']=='real')&(regression_df['regressor']=='linear')).values, :]
regression_df = regression_df.sort_values(['animal', 'day', 'cell_id'], axis=0).reset_index(drop=True)

# # add the match_id to the TC
# tc_whole['match_id'] = regression_df['match_id']

# save copies of the original for subselection
tc_whole_ori = tc_whole.copy()
regression_df_ori = regression_df.copy()

In [ ]:
# load the mine data
# define the base directory
base_path = r'Z:\test_mine'
# define the list of fields
field_list = ['correlations_trained', 'correlations_test', 'taylor_scores', 'taylor_true_change', 'taylor_full_prediction', 'taylor_by_predictor', 'nl_probs', 'me_scores', 'jacobians']

# get the files in the directory
files = os.listdir(base_path)
# allocate memory for the mine data
mine_list = []
mouse_date_list = []
cell_ids = []
# for all the files
for file in files[1:]:
    
    # get the mouse and day
    name = file.split('_')
    mouse = '_'.join(name[0:3])
    day = name[3]
    filepath = os.path.join(base_path, file)
    # load the file
    with h5py.File(filepath, mode='r') as f:    
        current_mine = [np.array(f[el]) for el in field_list]
        temp_ids = np.array(f['cell_ids'])
        temp_ids = [str(el).replace('[', '').replace(']', '').replace("'", '').split(' ') for el in temp_ids]
        # very ugly fix to get rid of the b
        temp_list = []
        for el in temp_ids:
            temp = [el2[1:] for el2 in el]
            temp_list.append(temp)
        temp_ids = temp_list
        cell_ids.append(temp_ids)
    # create the mine object
    current_mine = MineData(*current_mine)
    
    # store on a list
    mine_list.append(current_mine)
    mouse_date_list.append((mouse, day))

# load a single file to get the predictor columns variable (crappy crappy hacky code)
predictor_columns = variable_list
# # get the paths from the database using search_list
# all_paths, all_queries = fl.query_search_list()
# # print(all_paths)

# # data_list = []
# # load the data
# for path, queries in zip(all_paths[:1], all_queries[:1]):
    
#     data, _, _  = fl.load_preprocessing(path[:1], queries[:1])
# #     data_list.append(data)
    
# # print(data)
# predictor_columns = data[0].columns
# predictor_columns = [el for el in predictor_columns if (el not in ['motifs', 'mouse', 'datetime']) & ('cell_' not in el)]

In [ ]:
# create a dataframe with the relevant parts of the mine data
# get the number of predictors from MINE
predictor_number = len(predictor_columns)
print(predictor_number)
scores_number = ((predictor_number**2) - predictor_number)/2 +predictor_number

# allocate for the current trial
mine_cells = []
# for all date/mice
for current_mine, (mouse, date), ids in zip(mine_list, mouse_date_list, cell_ids):
    
    # for all the cells
    for cell, _ in enumerate(current_mine.correlations_trained):
#         print(current_mine.taylor_scores[cell, :, 0].shape)
#         print(mouse, date, ids[cell])
        taylor_scores = current_mine.taylor_scores[cell, :, 0].flatten()
        if taylor_scores.shape[0] < scores_number:
            taylor_scores = np.concatenate([taylor_scores, np.zeros((int(scores_number - taylor_scores.shape[0])))*np.nan], axis=0)
#         raise ValueError
        temp_list = [current_mine.correlations_test[cell], *taylor_scores, current_mine.nl_probabilities[cell], current_mine.mean_exp_scores[cell], mouse, date, ids[cell][0]]
#         print(temp_list)
#         raise ValueError
#         print(len(temp_list))
        mine_cells.append(np.array(temp_list))

# generate the column names
interaction_names = []
idx = 1
for name1 in predictor_columns:
    for name2 in predictor_columns[idx:]:
#         if name1 == name2:
#             continue
        interaction_names.append(f'{name1}_{name2}')
    idx += 1
taylor_columns = predictor_columns + interaction_names
columns = ['correlation_test', *taylor_columns, 'nl_probability', 'mean_exp_score', 'animal', 'day', 'cell_ids']
# turn the output into a dataframe
mine_cells = pd.DataFrame(mine_cells, columns=columns)
date_temp = []
for el in mine_cells.loc[:, 'day']:
    name = el.split('-')
    new_date = '_'.join([name[1], name[2], name[0]])
    date_temp.append(new_date)
mine_cells.loc[:, 'day'] = date_temp

In [ ]:
print(len(mouse_date_list))

In [ ]:
# tc_whole = tc_whole_ori
# regression_df = regression_df_ori

In [ ]:
# print(mine_cells['mouse_speed'])

cells = mine_cells['mouse_speed'].to_numpy().astype(float)
cells = cells[~np.isnan(cells)]
cells = np.log10(np.abs(cells))
# cells = cells[cells>-5]
print(cells.shape)
freq, location = np.histogram(cells, bins=100)

plot = hv.Scatter((location, freq))
plot.opts(width=600, height=600, tools=['hover'])
plot

In [ ]:
# match cells across TCs, regression and mine

# group by the animal and day 
for idx, ((mouse, day), data) in enumerate(mine_cells.groupby(['animal', 'day'])):
    

In [ ]:
# select only the cells that are present in regression, mine and tc

# allocate a list for the reordered MINE data
mine_accumulator = []
tc_accumulator = []
reg_accumulator = []
# and also an empty selection vector
selection_vector = np.zeros((tc_whole.shape[0]))
# cycle through the days and mice
for idx, ((mouse, day), data) in enumerate(mine_cells.groupby(['animal', 'day'])):
    # get the current mouse and day in the tuning curves
    temp_selection = ((tc_whole['animal'].values == mouse) & (tc_whole['day'].values == day))
    print(temp_selection)
    raise ValueError
#     # if there are cells in these conditions
#     if temp_selection.sum() > 0:
#         # check if the selection vector is empty
#         if selection_vector.sum() == 0:
#             # if yes, make the current vector the selection vector
#             selection_vector = temp_selection
#         else:
#             # otherwise, OR it with the existing one
#             selection_vector = selection_vector | temp_selection
    tc_accumulator.append(tc_whole.iloc[temp_selection, :])
    reg_accumulator.append(regression_df.iloc[temp_selection, :])
    print(data.columns)
    # also save the target mine fields in a preallocated list
    mine_accumulator.append(data[variable_list + ['animal', 'day', 'cell_ids', 'nl_probability', 'mean_exp_score', 'correlation_test']])        
    
#     print(mouse, day, data)
#     break
        
# tc_whole = tc_whole.iloc[selection_vector, :]
# regression_df = regression_df.iloc[selection_vector, :]
tc_whole = pd.concat(tc_accumulator)
regression_df = pd.concat(reg_accumulator)
mine_cells = pd.concat(mine_accumulator)
print(f'TC shape: {tc_whole.shape}, Reg shape: {regression_df.shape}, MINE shape: {mine_cells.shape}')
    


In [ ]:
print(mine_cells.columns)

In [ ]:
print(tc_whole.loc[:, ['cell_id', 'animal', 'day']], mine_cells.loc[:, ['cell_ids', 'animal', 'day']])

In [ ]:
# Prepare the array of indexes for comparative calculations

# get the tuning indexes
tc_index_columns = [el for el in tc_whole.columns if '_Qual_index' in el]
tc_index = tc_whole[tc_index_columns].copy()

tc_test_columns = [el for el in tc_whole.columns if '_Qual_test' in el]
tc_test = tc_whole[tc_test_columns].copy()

tc_index = tc_index * tc_test.values

# tc_index = mine_cells[variable_list]
regression_index = mine_cells[variable_list]
# print((tc_test.values!=0).sum())
# get the regression importances
# regression_index = regression_df.loc[:, variable_list].iloc[((regression_df['rvs']=='real')&(regression_df['regressor']=='linear')).values, :]

print(f'TC shape: {tc_index.shape}, Reg shape: {regression_index.shape}')
# combine them
all_indexes = pd.concat([tc_index, regression_index], axis=1)
all_columns = all_indexes.columns
all_indexes = all_indexes.to_numpy().astype(float)
# normalize
# all_indexes = preproc.StandardScaler().fit_transform(all_indexes)
all_indexes[np.isnan(all_indexes)] = 0

In [ ]:
%%time
# embed
reducer = umap.UMAP(min_dist=0.5, n_neighbors=30)
embedded_data = reducer.fit_transform(all_indexes[:, :11])

In [ ]:
# calculate labels

# define the target variable
target_parameter = 'mouse_speed'
perc = 99

column_idx = [idx for idx, el in enumerate(all_columns) if target_parameter in el]

idx0 = all_indexes[:, column_idx[0]].copy()
idx1 = all_indexes[:, column_idx[1]].copy()

# idx0 = np.abs(idx0)
# idx1 = np.abs(idx1)

# idx0 = (idx0 - idx0.min())/(idx0.max() - idx0.min())
# idx1 = (idx1 - idx1.min())/(idx1.max() - idx1.min())

# raw_labels = (idx0-idx1)/(idx0+idx1)
# label = np.abs(idx1)
# raw_labels = idx1
# raw_labels = np.abs(idx1)

# label = np.log(np.abs(idx1))
raw_labels = [idx0.copy(), idx1.copy()]
umap_list = []
for label in raw_labels:
    
    label[label>np.percentile(label, perc)] = np.percentile(label, perc)
    label[label<np.percentile(label, 100-perc)] = np.percentile(label, 100-perc)

    plot_data = np.concatenate([embedded_data, label.reshape((-1, 1))], axis=1)

    # plot the trials
    umap_plot = hv.Scatter(plot_data, vdims=['Dim 2', 'Parameter'], kdims=['Dim 1'])
    # umap_plot = hv.HexTiles(umap_data, kdims=['Dim 1', 'Dim 2'])
    umap_plot.opts(colorbar=True, color='Parameter', cmap='Spectral', tools=['hover'], alpha=1)
    umap_plot.opts(width=600, height=700, size=5)
    umap_list.append(umap_plot)
hv.Layout(umap_list).cols(2).opts(shared_axes=False)

In [ ]:


# ticks = [(idx+0.5, el) for idx, el in enumerate(variable_list)]
# plot = hv.Raster(all_indexes[:, :11].T)
# plot.opts(width=1200, height=600, colorbar=True, cmap='RdBu', tools=['hover'], yticks=ticks, xrotation=0, xlabel='')

In [ ]:
x = idx0.copy()
y = idx1.copy()

# x = np.abs(x)
# y = np.abs(y)
selection_vector = (x!=0)&(y!=0)
x = x[selection_vector]
y = y[selection_vector]

plot = hv.Scatter((x, y)).opts(xlabel='Tuning index', ylabel='Taylor metric', title=f'{stat.spearmanr(x, y)[0]:0.3f}')
plot.opts(width=600, height=600, tools=['hover'], ylim=(-1, 1.5))#, xlim=(0, 0.7))

In [ ]:
freq, loc = np.histogram(raw_labels, bins=30)

hv.Bars((loc, freq)).opts(width=600, xrotation=45)

In [ ]:
# explore the roi info

# # plot are distributions
# freq, edges = np.histogram(np.log10(regression_df['area'].to_numpy()), bins=40)
# areas = hv.Histogram((edges, freq), kdims='ROI area (log(px))')
# areas.opts(xrotation=45, width=400)

# # calculate the distance between matches
# # annotate a list to accumulate distances
# distance_list = []
# # run through all the mice
# for mouse_name, mouse_data in regression_df.groupby(['animal']):
#     for cell_name, cell_data in mouse_data.groupby(['match_id']):
#         # skip the singles
#         if cell_data.shape[0] == 1:
#             continue
#         # get the features
#         features = cell_data[['centroid_x', 'centroid_y']]

#         # get the distance matrix
#         distance_matrix = metrics.pairwise_distances(features.to_numpy())
# #         distance_list.append(np.diagonal(distance_matrix, offset=1))
#         distance_list.append(np.mean(distance_matrix, axis=0))
# # concatenate
# distance_list = np.hstack(distance_list)
# # calculate and plot the histogram
# freq_dist, edges_dist = np.histogram(distance_list, bins=40)
# distances = hv.Histogram((edges_dist, freq_dist), kdims='Cortical distance (px)')
# distances.opts(xrotation=45, width=400)

# (areas+distances).opts(shared_axes=False)

In [ ]:
# plot the roi centroid for matching pairs

# # initialize the plot list
# plot_list = []
# # define the number of matches to take per mouse
# target_matches = 10
# # generate a palette accordingly
# colors = palettes.brewer['Spectral'][target_matches]

# # run through all the mice
# for mouse_name, mouse_data in regression_df.groupby(['animal']):
    
#     # get the number of matched cells per id
#     match_numbers = mouse_data.groupby(['match_id'], as_index=False)['match_id'].count()

#     # exclude the non-matched
#     match_numbers = match_numbers.iloc[match_numbers['match_id'].to_numpy()>1, :]

#     # if there aren't enough matches, just use all
#     target_ids = np.array(match_numbers.index)
#     if match_numbers.shape[0] > target_matches:
#         target_ids = np.random.choice(target_ids, size=target_matches)
#     # initialize a list for this roi's plot
#     cell_plot = []
#     # for all the ids    
#     for idx, ids in enumerate(target_ids):

#         # get the data and plot
#         current_idx = mouse_data['match_id'].to_numpy() == ids
# #         current_data = mouse_data.loc[current_idx, ['centroid_x', 'centroid_y']]
# #         plot = hv.Scatter(current_data)
# #         cell_plot.append(plot)
#         current_data = mouse_data.loc[current_idx, ['bbox_left', 'bbox_top', 'bbox_width', 'bbox_height']]
#         # for all the cells
#         for index, cells in current_data.iterrows():
#             # assemble the bounding box for plotting
#             bbox = np.array([[cells['bbox_left'], cells['bbox_top']], [cells['bbox_left']+cells['bbox_width'], cells['bbox_top']],
#                             [cells['bbox_left']+cells['bbox_width'], cells['bbox_top']+cells['bbox_height']], [cells['bbox_left'], cells['bbox_top']+cells['bbox_height']], 
#                             [cells['bbox_left'], cells['bbox_top']]])
#             plot = hv.Curve(bbox)
#             plot.opts(color=colors[idx], width=400, height=400, title=mouse_name)
#             cell_plot.append(plot)
#     # overlay and store
#     plot_list.append(hv.Overlay(cell_plot))
# # plot
# hv.Layout(plot_list).cols(3)

In [ ]:
# sub-select cells

# # define a target feature to sort by
# target_feature = 'mouse_x'
# # define the percentage of cells to select
# target_percentage = 100

# # exclude cells that don't pass an area criterion

# # define the area criterion (based on the area histogram across the dataset)
# # area_min = 30
# # area_max = 300
# distance = 10

# # keep_vector = (regression_df_ori['area'].to_numpy() > area_min) & (regression_df_ori['area'].to_numpy() < area_max)

# # get the distances for the distance criterion
# # annotate a list to accumulate distances
# distance_list = []
# # run through all the mice
# for mouse_name, mouse_data in regression_df_ori.groupby(['animal']):
#     for cell_name, cell_data in mouse_data.groupby(['match_id']):
#         # skip the singles
#         if cell_data.shape[0] == 1:
#             average_distance = np.nan
#         else:
#             # get the features
#             features = cell_data[['centroid_x', 'centroid_y']]

#             # get the distance matrix
#             distance_matrix = metrics.pairwise_distances(features.to_numpy())
#     #         distance_list.append(np.diagonal(distance_matrix, offset=1))
#             average_distance = np.mean(distance_matrix, axis=0)
#         # assemble a dataframe
#         distance_df = cell_data[['animal', 'day', 'cell_id']].copy()
#         distance_df.loc[:, 'distance'] = average_distance
        
#         distance_list.append(distance_df)
# # concatenate
# distance_list = pd.concat(distance_list, axis=0)
# distance_list = distance_list.sort_values(['animal', 'day', 'cell_id'], axis=0).reset_index(drop=True)

# # threshold
# keep_vector = (keep_vector) & (distance_list['distance'].to_numpy() < 10)
# # replace in the data
# regression_df = regression_df_ori.iloc[keep_vector, :]
# tc_whole = tc_whole_ori.iloc[keep_vector, :]

# # leave only cells with matches

# # allocate the list for accumulation
# accumulation_list = []
# # also allocate a list for the cell coordinates to save
# idx_list = []
# # run through all the mice
# for mouse_name, mouse_data in regression_df.groupby(['animal']):
    
#     for cell_name, cell_data in mouse_data.groupby(['match_id']):
        
#         # skip the singles
#         if cell_data.shape[0] == 1:
#             continue
#         # accumulate the cells
#         accumulation_list.append(cell_data)
#         # accumulate the idx
#         idx_list.append(cell_data.index.to_numpy())
# # concatenate them
# regression_df = pd.concat(accumulation_list)
# # also sub-select from the TCs
# tc_whole = tc_whole.loc[np.hstack(idx_list), :]

# # sort by a target feature

# # get the actual target number
# target_number = int(np.round(tc_whole.shape[0]*target_percentage/100))

# # get the feature sorting
# feature = tc_whole.loc[:, target_feature+'_Cons_index'].to_numpy()
# # feature = regression_df_ori.loc[:, target_feature].to_numpy()
# feature_idx = np.argsort(feature)
# sorted_feature = feature[feature_idx]
# # exclude the nans
# feature_idx = feature_idx[~np.isnan(sorted_feature)]
# feature_idx = feature_idx[-target_number:]

# # get the cells from both 
# tc_whole = tc_whole.iloc[feature_idx, :]
# regression_df = regression_df.iloc[feature_idx, :]
# # resort
# tc_whole = tc_whole.sort_values(['animal', 'day', 'cell_id'], axis=0).reset_index(drop=True)
# regression_df = regression_df.sort_values(['animal', 'day', 'cell_id'], axis=0).reset_index(drop=True)
# print(feature[feature_idx])

In [ ]:
# calculate the correlation matrix
# # get the columns for both methods
# resp_columns = [el for el in tc_whole.columns if 'Resp_index' in el]
# cons_columns = [el for el in tc_whole.columns if 'Cons_index' in el]
# qual_columns = [el for el in tc_whole.columns if 'Qual_index' in el]
# # get the TCs
# tc_resp = tc_whole.loc[:, resp_columns].to_numpy()
# tc_cons = tc_whole.loc[:, cons_columns].to_numpy()
# tc_qual = tc_whole.loc[:, qual_columns].to_numpy()
# tc_indexes = tc_qual

# allocate memory for the matrices
matrix_list = []

# for all the mice
for mouse_name, mouse_data in tc_whole.groupby(['animal']):
    # get the relevant columns from the TCs
    target_columns = [el for el in tc_whole.columns if 'Qual_index' in el]
    tc_matrix = mouse_data[target_columns].to_numpy()
    test_columns = [el for el in tc_whole.columns if 'Qual_test' in el]
    tc_test = mouse_data[test_columns].to_numpy()
    tc_matrix = tc_matrix * tc_test
# for mouse_name, mouse_data in mine_cells.groupby(['animal']):
#     tc_matrix = mouse_data.loc[:, variable_list].to_numpy()
    
    # get the regressions
#     regression_matrix = mine_cells.iloc[mine_cells.loc[:, 'animal'].to_numpy() == mouse_name, :]
    regression_matrix = regression_df.iloc[regression_df.loc[:, 'animal'].to_numpy() == mouse_name, :]
    regression_matrix = regression_matrix.loc[:, variable_list].to_numpy()
    
    print(tc_matrix.shape, regression_matrix.shape)

# take the abs
# tc_indexes = np.abs(tc_indexes)
# regression_matrix = np.abs(regression_matrix)

    # calculate correlation
    correlation_matrix, pvalue_matrix = stat.spearmanr(tc_matrix, regression_matrix, nan_policy='omit')
    
    matrix_list.append(correlation_matrix)

matrix_list = np.array(matrix_list)

correlation_average = np.mean(matrix_list, axis=0)[len(variable_list):, :len(variable_list)]
correlation_error = stat.sem(matrix_list, axis=0)[len(variable_list):, :len(variable_list)]
# print(tc_indexes.shape)
# print(regression_matrix.shape)
print(matrix_list.shape)

# Interaction correlation plot  

In [ ]:
# plot the correlation matrix
# plot_list = []

# ticks = [(idx+0.5, el[:-11]) for idx, el in enumerate(resp_columns+resp_columns)]
# raster = hv.Raster(correlation_matrix, kdims=['Regression', 'TCs'])
# raster.opts(width=1000, height=800, yticks=ticks, xticks=ticks, xrotation=45, colorbar=True, cmap='RdBu', clim=(-1, 1), tools=['hover'])
# plot_list.append(raster)

# ticks = [(idx+0.5, el[:-11]) for idx, el in enumerate(resp_columns)]
# raster = hv.Raster(correlation_matrix[len(resp_columns):, :len(resp_columns)], kdims=['Regression', 'TCs'])
# raster.opts(width=1000, height=800, yticks=ticks, xticks=ticks, xrotation=45, colorbar=True, cmap='RdBu', clim=(-1, 1), tools=['hover'])
# plot_list.append(raster)

ticks = [(idx+0.5, label_dict[el]) for idx, el in enumerate(variable_list)]
# ticks = [(idx+0.5, idx) for idx, el in enumerate(variable_list)]
plot_matrix = correlation_average.copy()
# plot_matrix = np.tril(plot_matrix, k=0)
plot_matrix[plot_matrix==0] = np.nan
# hv.Raster(correlation_matrix)
raster = hv.Raster(plot_matrix)
# format the plot
raster.opts(width=950, height=800, yticks=ticks, xticks=ticks, colorbar=True, cmap='RdBu', clim=(-1, 1), xrotation=45, 
            xlabel='Importance', ylabel='Tuning')

# assemble the file name
save_name = os.path.join(save_path, '_'.join((target_document, 'Interaction_correlation')) + '.png')
# save the figure
fig = fp.save_figure(raster, save_name, fig_width=15, dpi=1200, fontsize=target_document, target='screen')

# ticks = [(idx+0.5, el[:-11]) for idx, el in enumerate(resp_columns)]
# raster = hv.Raster(correlation_matrix[:len(resp_columns), :len(resp_columns)], kdims=['TCs', 'TCs'])
# raster.opts(width=1000, height=800, yticks=ticks, xticks=ticks, xrotation=45, colorbar=True, cmap='RdBu', clim=(-1, 1), tools=['hover'])
# plot_list.append(raster)

# ticks = [(idx+0.5, el[:-11]) for idx, el in enumerate(resp_columns)]
# raster = hv.Raster(correlation_matrix[len(resp_columns):, len(resp_columns):], kdims=['Regression', 'Regression'])
# raster.opts(width=1000, height=800, yticks=ticks, xticks=ticks, xrotation=45, colorbar=True, cmap='RdBu', clim=(-1, 1), tools=['hover'])
# plot_list.append(raster)

# hv.Layout(plot_list).cols(1).opts(shared_axes=False)
# print(ticks)

# Interaction diagonal plot 

In [ ]:
# plot the diagonal of the interaction matrix across animals
importlib.reload(fp)
# get the diagonals
diagonal_average = np.diagonal(correlation_average)
diagonal_error = np.diagonal(correlation_error)
# get the x coordinate
x = [label_dict[el] for el in variable_list]

# plot
dots = hv.Scatter((x, diagonal_average))
dots.opts(size=10, ylabel='Tuning-to-Importance CC', xlabel='')
error = hv.ErrorBars((x, diagonal_average, diagonal_error))
error.opts(width=600, height=600, xrotation=45, line_width=1)

plot = dots*error
# assemble the file name
save_name = os.path.join(save_path, '_'.join((target_document, 'Interaction_diagonal')) + '.png')
# save the figure
plot = fp.save_figure(plot, save_name, fig_width=10, dpi=1200, fontsize=target_document, target='screen')

# plot

# Top cells scatter

In [ ]:
# plot the top and worst cells for each feature

# allocate memory for the plot list
plot_list = []
# define the percentages to plot
percentage_target = 10
number_target = int(np.round(tc_whole.shape[0]*(percentage_target/100)))
# for all the features
for feature in variable_list[:4]:
    # get the TC and regression data
#     print(tc_whole.columns)
    tc_resp = tc_whole.loc[:, feature+'_Resp_index'].to_numpy().copy()
    tc_cons = tc_whole.loc[:, feature+'_Cons_index'].to_numpy().copy()
    tc_quals = tc_whole.loc[:, feature+'_Qual_index'].to_numpy().copy()
    tc_data = tc_quals
#     tc_data = np.abs(tc_data)
#     tc_data[tc_data > 100] = 0
    regression_data = regression_df.loc[:, feature].to_numpy().copy()
    
    # exclude nan values
    keep_vector = ~np.isnan(tc_data)
    tc_data = tc_data[keep_vector]
    regression_data = regression_data[keep_vector]
    # get the indexes of the top and bottom n percent
    idx = np.argsort(regression_data)
    
    # get the top and bottom
    tc_top = tc_data[idx[-number_target:]]
    regression_top = regression_data[idx[-number_target:]]
    
    tc_bottom = tc_data[idx[:number_target]]
    regression_bottom = regression_data[idx[:number_target]]
    
#     # artificially sort them
#     tc_top.sort()
#     regression_top.sort()
    
    correlation = stat.spearmanr(tc_top, regression_top)[0]
    print(correlation)
    
    # plot
    top_scatter = hv.Scatter((regression_top, tc_top), kdims=['Imp. index'], vdims='TC index')
#     top_scatter.opts(xrotation=45, title=f'{feature}: {correlation:0.2f}', width=300, height=300)
    top_scatter.opts(xrotation=45, frame_width=300, frame_height=300, title=label_dict[feature], yticks=3, xticks=3)
    bottom_scatter = hv.Scatter((tc_bottom, regression_bottom))
    
    # assemble the file name
    save_name = os.path.join(save_path, '_'.join((target_document, 'Top_weights', feature)) + '.png')
    # save the figure
    top_scatter = fp.save_figure(top_scatter, save_name, fig_width=2.8, dpi=1200, fontsize=target_document, target='screen')
    
    plot_list.append(top_scatter)
    
# hv.Layout(plot_list).cols(4).opts(shared_axes=False)


In [ ]:
# normalize the weights between 0-1

regression_normalized = regression_df.copy()

# regression_normalized.loc[:, variable_list] = (regression_normalized.loc[:, variable_list] - regression_normalized.loc[:, variable_list].min(axis=0)) / \
#     (regression_normalized.loc[:, variable_list].max(axis=0) - regression_normalized.loc[:, variable_list].min(axis=0))


In [ ]:
# plot the cell matching based info (can use match_id for both methods since they are already sorted)

print(regression_normalized.columns)
# allocate memory for the correlations
correlation_list = []
# run through all the mice
for mouse_name, mouse_data in regression_normalized.groupby(['animal']):
    # if the mouse is empty, skip
#     print(mouse_data.shape)
#     if mouse_data.shape[0] == 0:
#         continue
    #     print(mouse)
    mouse_list = []
    
    for cell_name, cell_data in mouse_data.groupby(['match_id']):
        
        # skip the singles
        if cell_data.shape[0] == 1:
            continue
        # get the features
        features = cell_data[variable_list].to_numpy()
        # calculate the correlation matrix
        correlation = stat.spearmanr(features.T, nan_policy='omit')[0]
        # get the average of the off diagonal elements
#         print(correlation)
        
        if isinstance(correlation, np.ndarray):
            triu = np.triu(correlation, k=1)
            average = np.mean(triu[triu != 0])
        else:
            average = correlation
#         print(type(average))
#         average = np.float(average)
        # store
        mouse_list.append([mouse_name, cell_name, average])
#         print(type(mouse_list[-1][-1]))
#         print(np.vstack(mouse_list))
#         raise ValueError
    # if no matches were found, skip
    if len(mouse_list) == 0:
        continue
    correlation_list.append(pd.DataFrame(np.vstack(mouse_list), columns=['animal', 'match_id', 'correlation']))
#     print(correlation_list[0])
#     raise ValueError
    
correlation_list = pd.concat(correlation_list)
correlation_list['correlation'] = correlation_list['correlation'].to_numpy().astype(float)
print(correlation_list)

In [ ]:
# plot the distribution of correlations

plot_list = []
# # for all the mice
# for mouse_name, mouse_data in correlation_list.groupby(['animal']):
#     plot = hv.Scatter(mouse_data['correlation'])
# plot_array = correlation_list[['animal', 'correlation']].copy().reset_index(drop=True)
plot_array = correlation_list
# plot_array[plot_array.isna()] = 0
# print(type(plot_array.iloc[1, 1]))
# print(type(float(plot_array.iloc[1, 1])))
# plot_array[np.isinf(plot_array)] = 0
box = hv.BoxWhisker(plot_array, ['animal'], ['correlation'])
box.opts(width=800, height=600, xrotation=45)
box

In [ ]:
# calculate averages per mouse

# allocate memory for the correlations
correlation_interval = []
# run through all the mice
for mouse_name, mouse_data in regression_normalized.groupby(['animal']):
    
    mouse_list = []    
    random_list = []
    for cell_name, cell_data in mouse_data.groupby(['match_id']):
        # skip the singles
        if cell_data.shape[0] == 1:
            continue
        # get the features
        features = cell_data[variable_list]
        
        # calculate the correlation matrix
        correlation = stat.spearmanr(features.T, nan_policy='omit')[0]
        
        # get a set of random cells of this size
        random_idx = np.random.randint(0, mouse_data.shape[0]-1, cell_data.shape[0])
        random_cells = mouse_data.iloc[random_idx, :]
        
        random_correlation = stat.spearmanr(random_cells[variable_list].T, nan_policy='omit')[0]
        
        if isinstance(random_correlation, float) & isinstance(correlation, np.ndarray):
            random_correlation = np.zeros_like(correlation)*np.nan
        elif isinstance(correlation, float) & isinstance(random_correlation, np.ndarray):
            random_correlation = np.nan
        # get the delta days
        day_data = cell_data.loc[:, 'day'].to_numpy()
        delta_days = [(el-day_data[0]) for el in day_data]
        delta_days = (delta_days/np.timedelta64(1, 'D')).astype(int)

        # get the average of the off diagonal elements
        if isinstance(correlation, np.ndarray):
            # for all the off diagonals
            for el in np.arange(1, correlation.shape[0]):
                diag = np.diagonal(correlation, offset=el)
                average = np.mean(diag)
                
                random_diag = np.diagonal(random_correlation, offset=el)
                random_average = np.mean(random_diag)
                # store
                mouse_list.append([mouse_name, cell_name, delta_days[el], average])   
                random_list.append([mouse_name+'_random', cell_name, delta_days[el], random_average])   
        else:
            average = correlation
            random_average = random_correlation
            # store
            mouse_list.append([mouse_name, cell_name, delta_days[1], average])
            random_list.append([mouse_name+'_random', cell_name, delta_days[1], random_average])
    # if no matches were found, skip
    if len(mouse_list) == 0:
        continue
    correlation_interval.append(pd.DataFrame(np.vstack(mouse_list), columns=['animal', 'match_id', 'interval', 'correlation']))
    correlation_interval.append(pd.DataFrame(np.vstack(random_list), columns=['animal', 'match_id', 'interval', 'correlation']))

    
correlation_interval = pd.concat(correlation_interval)
correlation_interval['correlation'] = correlation_interval['correlation'].to_numpy().astype(float)

print(correlation_interval.shape)

In [ ]:
# plot the correlations per animal

# allocate a list for the plots
plot_list = []
# calculate the mean and sem
grouped_data = correlation_interval.groupby(['animal'], as_index=False)[['interval', 'correlation']]

# for all the animals
for mouse_name, mouse_data in grouped_data:
    
    mouse_mean = mouse_data.groupby(['interval'], as_index=False)['correlation'].mean()
    mouse_mean.loc[:, 'interval'] = mouse_mean.loc[:, 'interval'].astype(float)
    mouse_mean = mouse_mean.sort_values(['interval'])
    
    mouse_sem = mouse_data.groupby(['interval'], as_index=False)['correlation'].sem()
    mouse_sem.loc[:, 'interval'] = mouse_sem.loc[:, 'interval'].astype(float)
    mouse_sem = mouse_sem.sort_values(['interval'])
    
    mean_plot = hv.Curve((mouse_mean['interval'], mouse_mean['correlation']), kdims=['Interval'], vdims='Correlation', label=mouse_name)
    sem_plot = hv.Spread((mouse_mean['interval'], mouse_mean['correlation'], mouse_sem['correlation']))
    mean_plot.opts(height=600, width=800, xrotation=45)
    
    plot_list.append(mean_plot*sem_plot)
hv.Overlay(plot_list)

In [ ]:
# calculate averages per feature

# allocate memory for the correlations
feature_diffs = {}

# for all the features
for feature in variable_list:
    feature_df = []
    # run through all the mice
    for mouse_name, mouse_data in regression_normalized.groupby(['animal']):
        mouse_list = []
        random_list = []
        for cell_name, cell_data in mouse_data.groupby(['match_id']):
            # skip the singles
            if cell_data.shape[0] == 1:
                continue
            # get the features
            features = cell_data[feature]

            # get the distance matrix
            distances = metrics.pairwise_distances(features.to_numpy().reshape(-1, 1))
#             distances[distances == 0] = np.nan
#             distances = 1 - distances
    
            # get a set of random cells of this size
            random_idx = np.random.randint(0, mouse_data.shape[0]-1, cell_data.shape[0])
            random_cells = mouse_data.iloc[random_idx, :]
            random_distances = metrics.pairwise_distances(random_cells[feature].to_numpy().reshape(-1, 1))
#             random_distances = 1 - random_distances
            
            # get the delta days
            day_data = cell_data.loc[:, 'day'].to_numpy()
            delta_days = [(el-day_data[0]) for el in day_data]
            delta_days = (delta_days/np.timedelta64(1, 'D')).astype(int)
            # get the average of the off diagonal elements
            if isinstance(distances, np.ndarray):
                # for all the off diagonals
                for el in np.arange(1, distances.shape[0]):
                    diag = np.diagonal(distances, offset=el)
                    average = np.mean(diag)
                    mouse_list.append([mouse_name, cell_name, delta_days[el], average])
                    
                    random_diag = np.diagonal(random_distances, offset=el)
                    random_average = np.mean(random_diag)
                    random_list.append([mouse_name+'_random', 'random', delta_days[el], random_average])

            else:
                average = distances
                random_average = random_distances

                # store
                mouse_list.append([mouse_name, cell_name, delta_days[1], average])
                random_list.append([mouse_name+'_random', 'random', delta_days[1], random_average])
        # if not matches were found, skip
        if len(mouse_list) == 0:
            continue
        feature_df.append(pd.DataFrame(np.vstack(mouse_list), columns=['animal', 'match_id', 'interval', 'similarity']))
        feature_df.append(pd.DataFrame(np.vstack(random_list), columns=['animal', 'match_id', 'interval', 'similarity']))

    feature_df = pd.concat(feature_df)
    feature_df['similarity'] = feature_df['similarity'].to_numpy().astype(float)
    # store for the particular feature
    feature_diffs[feature] = feature_df

print(feature_diffs[feature].shape)

In [ ]:
# plot the per-animal and per-feature distances
main_list = []

# for all the features
for feature in variable_list:
    plot_list = []
    # get the dataframe
    current_df = feature_diffs[feature]
    
    # calculate the mean and sem
    grouped_data = current_df.groupby(['animal'], as_index=False)[['interval', 'similarity']]

    # for all the animals
    for mouse_name, mouse_data in grouped_data:

        mouse_mean = mouse_data.groupby(['interval'], as_index=False)['similarity'].mean()
        mouse_mean.loc[:, 'interval'] = mouse_mean.loc[:, 'interval'].astype(float)
        mouse_mean = mouse_mean.sort_values(['interval'])

        mouse_sem = mouse_data.groupby(['interval'], as_index=False)['similarity'].sem()
        mouse_sem.loc[:, 'interval'] = mouse_sem.loc[:, 'interval'].astype(float)
        mouse_sem = mouse_sem.sort_values(['interval'])

        mean_plot = hv.Curve((mouse_mean['interval'], mouse_mean['similarity']), kdims=['Interval'], vdims='Similarity')
        sem_plot = hv.Spread((mouse_mean['interval'], mouse_mean['similarity'], mouse_sem['similarity']))
        mean_plot.opts(height=400, width=400, xrotation=45, title=feature)

        plot_list.append(mean_plot*sem_plot)
    main_list.append(hv.Overlay(plot_list))

hv.Layout(main_list).cols(4).opts(shared_axes=False)

In [ ]:
# plot across intervals

# plot the per-animal and per-feature distances
main_list = []
# define the target interval
target_interval = '1'
# for all the features
for feature in variable_list:
    # get the dataframe
#     current_df = feature_diffs[feature]
#     current_df['feature'] = feature
    # get the dataframe
    current_df = feature_diffs[feature]
    current_real = current_df[(current_df['match_id'] != 'random') & (current_df['interval'] == target_interval)]
    current_random = current_df[(current_df['match_id'] == 'random') & (current_df['interval'] == target_interval)]
    current_real['feature'] = feature
    current_random['feature'] = feature+'_random'
    
    current_real['similarity'] = current_real['similarity'].fillna(0)
    current_random['similarity'] = current_random['similarity'].fillna(0)
#     print((current_df['interval'] ))
    print(f"{feature}: {stat.mannwhitneyu(current_real['similarity'], current_random['similarity'])[1]*len(variable_list):0.4f}")

    # store
    main_list.append(current_real)
    main_list.append(current_random)

    # store
    main_list.append(current_df)

# turn into a dataframe
main_list = pd.concat(main_list)

box = hv.BoxWhisker(main_list, ['feature'], ['similarity'])
box.opts(width=1200, height=600, xrotation=45, ylabel='distance')
box
#         mean_plot = hv.Curve((mouse_mean['interval'], mouse_mean['correlation']))
#         sem_plot = hv.Spread((mouse_mean['interval'], mouse_mean['correlation'], mouse_sem['correlation']))
#         mean_plot.opts(height=400, width=400, xrotation=45, title=feature)

#         plot_list.append(mean_plot*sem_plot)
#     main_list.append(hv.Overlay(plot_list))

# hv.Layout(main_list).cols(4).opts(shared_axes=False)

In [ ]:
# get the tc similarities by correlating the actual TCs

# print(tc_whole.columns[:50])
# allocate the output
tc_interval = {}
# for all the features
for feature in variable_list:
    # get the relevant columns
    current_columns = [el for el in tc_whole.columns if ('bin' in el) & (feature in el)]
#     print(current_columns)
#     raise ValueError
    current_df = tc_whole[['day', 'animal', 'match_id'] + current_columns]
    
    feature_df = []
    # run through all the mice
    for mouse_name, mouse_data in current_df.groupby(['animal']):
        mouse_list = []
        random_list = []
        index_list = []
        for match_id, cell_data in mouse_data.groupby(['match_id']):
            # skip the singles
            if cell_data.shape[0] < 3:
                continue
            # calculate the correlation
#             correlation = stat.spearmanr(cell_data[current_columns].T, nan_policy='omit')[0]
            current_cells = cell_data[current_columns].to_numpy()
            current_cells[np.isnan(current_cells)] = 0    
            correlation = metrics.pairwise_distances(current_cells)

            
            # get a set of random cells of this size
            random_idx = np.random.randint(0, mouse_data.shape[0]-1, cell_data.shape[0])
            random_cells = mouse_data.iloc[random_idx, :]
            random_cells = random_cells[current_columns].to_numpy()
            random_cells[np.isnan(random_cells)] = 0    
#             random_correlation = stat.spearmanr(random_cells[current_columns].T, nan_policy='omit')[0]
            random_correlation = metrics.pairwise_distances(random_cells)
            
            if isinstance(random_correlation, float) & isinstance(correlation, np.ndarray):
                random_correlation = np.zeros_like(correlation)*np.nan
            elif isinstance(correlation, float) & isinstance(random_correlation, np.ndarray):
                random_correlation = np.nan
            
            # get the delta days
            day_data = cell_data.loc[:, 'day'].to_numpy()
            delta_days = [(el-day_data[0]) for el in day_data]
            delta_days = (delta_days/np.timedelta64(1, 'D')).astype(int)
            # get the average of the off diagonal elements
            if isinstance(correlation, np.ndarray):
                # for all the off diagonals
                for el in np.arange(1, correlation.shape[0]):
                    diag = np.diagonal(correlation, offset=el)
                    average = np.mean(diag)
                    
                    random_diag = np.diagonal(random_correlation, offset=el)
                    random_average = np.mean(random_diag)
                    # store
                    mouse_list.append([mouse_name, match_id, delta_days[el], average])      
                    random_list.append([mouse_name+'_random', 'random', delta_days[el], random_average])      
            else:
                average = correlation
                random_average = random_correlation
                # store
                mouse_list.append([mouse_name, match_id, delta_days[1], average])
                random_list.append([mouse_name+'_random', 'random', delta_days[1], random_average])

        if len(mouse_list) == 0:
            continue
        feature_df.append(pd.DataFrame(np.vstack(mouse_list), columns=['animal', 'match_id', 'interval', 'correlation']))
        feature_df.append(pd.DataFrame(np.vstack(random_list), columns=['animal', 'match_id', 'interval', 'correlation']))
    
    
    feature_df = pd.concat(feature_df)
    feature_df['correlation'] = feature_df['correlation'].to_numpy().astype(float)
    
    # store for the particular feature
    tc_interval[feature] = feature_df

print(tc_interval[feature].shape)
    

In [ ]:
# plot the tc correlations as a function of feature
main_list = []

# for all the features
for feature in variable_list:
    plot_list = []
    # get the dataframe
    current_df = tc_interval[feature]
    
    # calculate the mean and sem
    grouped_data = current_df.groupby(['animal'], as_index=False)[['interval', 'correlation']]

    # for all the animals
    for mouse_name, mouse_data in grouped_data:

        mouse_mean = mouse_data.groupby(['interval'], as_index=False)['correlation'].mean()
        mouse_mean.loc[:, 'interval'] = mouse_mean.loc[:, 'interval'].astype(float)
        mouse_mean = mouse_mean.sort_values(['interval'])

        mouse_sem = mouse_data.groupby(['interval'], as_index=False)['correlation'].sem()
        mouse_sem.loc[:, 'interval'] = mouse_sem.loc[:, 'interval'].astype(float)
        mouse_sem = mouse_sem.sort_values(['interval'])
        
        mean_plot = hv.Curve((mouse_mean['interval'], mouse_mean['correlation']), kdims=['Interval'], vdims='Correlation')
        sem_plot = hv.Spread((mouse_mean['interval'], mouse_mean['correlation'], mouse_sem['correlation']))
        if 'random' in mouse_name:
            color = 'k'
        else:
            color = 'r'    
        mean_plot.opts(height=400, width=400, xrotation=45, title=feature, color=color)
        sem_plot.opts(height=400, width=400, xrotation=45, title=feature, color=color)

        plot_list.append(mean_plot*sem_plot)
    main_list.append(hv.Overlay(plot_list))

hv.Layout(main_list).cols(4).opts(shared_axes=False)

In [ ]:
# plot the per-animal and per-feature distances
main_list = []

# define the target interval
target_interval = '1'
# for all the features
for feature in variable_list:
    # get the dataframe
    current_df = tc_interval[feature]
    current_real = current_df[(current_df['match_id'] != 'random') & (current_df['interval'] == target_interval)]
    current_random = current_df[(current_df['match_id'] == 'random') & (current_df['interval'] == target_interval)]
    current_real['feature'] = feature
    current_random['feature'] = feature+'_random'
    
    current_real['correlation'] = current_real['correlation'].fillna(0)
    current_random['correlation'] = current_random['correlation'].fillna(0)
    
    print(f"{feature}: {stat.mannwhitneyu(current_real['correlation'], current_random['correlation'])[1]*len(variable_list):0.4f} {current_real.shape[0]}")
#     # calculate the mean and sem
#     data_mean = current_df.loc[:, 'correlation'].mean().to_numpy()
#     data_sem = current_df.loc[:, 'correlation'].sem().to_numpy()
    # store
    main_list.append(current_real)
    main_list.append(current_random)

# turn into a dataframe
main_list = pd.concat(main_list)

box = hv.BoxWhisker(main_list, ['feature'], ['correlation'])
box.opts(width=1000, height=600, xrotation=45, ylabel='Distance')
box

# Tuning curve examples 

In [ ]:
# plot top tuning curves
importlib.reload(fp)
# define the target variable
target_variables = ['mouse_x', 'mouse_speed', 'cricket_0_mouse_distance']

# get the tc ranges
tc_ranges = processing_parameters.tc_params
# target_variable = 'mouse_speed'
# target_variable = 'cricket_0_mouse_distance'
# define the number of cells to grab per animal
target_number = 1
# define how to select the TCs
selection_mode = 'top'

# allocate a list for the plots
plot_list = []
# for all the target variables
for target_variable in target_variables:
    # for all the mice
    for mouse_name, mouse_data in regression_df_ori.groupby(['animal']):
        # get the top weighted cell indexes

        # get their tuning curves
        mouse_tcs = tc_whole_ori[(tc_whole_ori['animal'] == mouse_name)]

    #     target_sorter = mouse_data[target_variable].to_numpy()
        target_sorter = mouse_tcs[target_variable+'_Qual_index'].to_numpy()
        top_weight_idx = np.argsort(target_sorter)
        top_weight = np.sort(target_sorter)

        # exclude nans
        keep_vector = ~np.isnan(top_weight)
        top_weight_idx = top_weight_idx[keep_vector]
        top_weight = top_weight[keep_vector]
        # randomize
#         np.random.shuffle(top_weight_idx)
        if selection_mode == 'top':
            # take the top n
            top_weight_idx = top_weight_idx[-target_number:]
            top_weight = top_weight[-target_number:]
        elif 'bottom':
            # take the bottom n
            top_weight_idx = top_weight_idx[:target_number]
            top_weight = top_weight[:target_number]
        else:
            # randomize
            continue


        tc_columns = [el for el in mouse_tcs.columns if (target_variable in el) & ('bin' in el)]
        tc_quals = mouse_tcs[target_variable+'_Qual_index'].to_numpy()
        tc_quals = tc_quals[top_weight_idx]
        mouse_tcs = mouse_tcs[tc_columns]
        mouse_tcs = mouse_tcs.iloc[top_weight_idx, :]
        
        # define the x ticks
        curr_range = tc_ranges[target_variable]
        curr_range = np.linspace(curr_range[0], curr_range[1], 20)
        xticks = [(int(el), int(curr_range[el])) for el in np.arange(5, 20, 5)]
        
#         print(xticks)
#         raise ValueError
        # for all the tcs
        for idx1, (idx, tc) in enumerate(mouse_tcs.iterrows()):
            # get the day
    #         current_day = str(mouse_data.iloc[top_weight[idx1], 'day'])
            # plot
            plot = hv.Curve((np.arange(tc.shape[0]), tc.to_numpy()), kdims='Variable', vdims='Signal')
#             plot.opts(width=400, height=600, yaxis=None, xlabel=' '.join((label_dict[target_variable], units_dict[target_variable])), xticks=xticks)
            plot.opts(width=400, height=400, yaxis=None, xticks=xticks, xlabel='')

            save_name = os.path.join(save_path, '_'.join((target_document, 'TC_ex', target_variable, mouse_name, str(idx))) + '.png')
            # save the figure
            fig = fp.save_figure(plot, save_name, fig_width=2, dpi=1200, fontsize='small', target='save')#, display_factor=0.1)
            plot_list.append(plot)
    
#     print(mouse_tcs)
#     raise ValueError
# hv.Layout(plot_list).opts(shared_axes=True).cols(target_number)

In [ ]:
# visualize single cell TCs based on matching

# define the target feature
target_feature = 'mouse_speed'

# define the number of groups to take
target_number = 20

# get only the non-random data
# print(tc_interval[target_feature]['match_id'] != 'random')
tc_nonrandom = tc_mean[target_feature][tc_mean[target_feature]['match_id'] != 'random']
# get the top N groups for the target feature
top_idx = np.argsort(tc_nonrandom['distance'].to_numpy())[:target_number]
# print(np.sort(tc_nonrandom['correlation'].to_numpy())[-target_number:])
# raise ValueError
top_groups = tc_nonrandom.iloc[top_idx, :]

# initialize the plot list
plot_list = []
# plot the corresponding TCs
for idx, group in top_groups.iterrows():
    # get the match id and mouse
    target_mouse = group['animal']
    target_id = float(group['match_id'])
    # get the individual tuning curves
    target_cells = tc_whole[(tc_whole['animal']==target_mouse) & (tc_whole['match_id']==target_id)]
    print(target_mouse, target_id, target_cells.shape, group['distance'])
    # get the relevant columns
    target_columns = [el for el in tc_whole.columns if ('bin' in el) & (target_feature in el)]
    target_cells = target_cells[target_columns]
    x = np.arange(target_cells.shape[1])
    # plot
    # accumulate the tuning curves
    tc_list = []
    # for all the cells
    for cell_idx, cell in target_cells.iterrows():
        plot = hv.Curve((x, cell.to_numpy()), kdims=['Bin'], vdims='Tuning')
        plot.opts(height=300, width=300)
        tc_list.append(plot)
    # store
    plot_list.append(hv.Overlay(tc_list))
# plot the layout
layout = hv.Layout(plot_list).opts(shared_axes=False)
layout